# Celebrimbor build eval

Scope at infantry and arty combat ability plus dev

In [1]:
import yaml
from collections import Counter, defaultdict
import itertools
from math import comb
from tqdm.notebook import tqdm
import random
from functools import lru_cache

In [2]:
%load_ext autoreload
%autoreload 2

from lib.model import *
from lib.build_tools import *
from lib.loader_tools import *


## Load data

In [3]:
IDEAS = load_ideas([
    'data/gypsy_transformed/ideas.yaml', 
    'data/gypsy_transformed/ideas_flogi.yaml', 
    'data/gypsy_transformed/ideas_flogi_relig.yaml'
])

POLICIES = load_policies([
    'data/gypsy_transformed/policies.yaml'
])

In [4]:
def print_build(build: Build, effects: list[str]):
    print('--------------------------------------------------------------------')
    print(f"Build - score: {build.score} - ideas: {build.ideas}")
    for effect in effects:
        print(f"\t{effect}: {build.total_effect[effect]:.2f}")

def get_ideas_effect(ideas: tuple[str]):
    total_effect = Counter()
    for idea in ideas:
        total_effect.update(IDEAS[idea].effect)
        
    return total_effect

def compute_build(ideas: tuple[str], base_policy_slots=4, debug=False) -> Build:
    ideas_effect = get_ideas_effect(ideas)

    available_policies = get_available_policies(ideas, POLICIES)
    max_policy_slots = get_max_policy_slots(ideas_effect, base_policy_slots)

    adm_max_policy, dip_max_policy, mil_max_policy = get_max_policy_slots(ideas_effect, base_policy_slots)
    micro_management_policies_effect = get_micro_management_policies_effect(ideas, available_policies, max_policy_slots)
    war_policies_effect = get_war_policies_effect(ideas, MILITARY_WEIGHTS, available_policies, max_policy_slots)

    total_effect = Counter()
    total_effect.update(ideas_effect)
    total_effect.update(micro_management_policies_effect)
    total_effect.update(war_policies_effect)


    
    return Build(
        ideas=ideas, 
        score=score(total_effect, [COUNTRY_WEIGHTS, MILITARY_WEIGHTS], debug=debug),
        total_effect=total_effect,
        war_policies_effect=war_policies_effect,
    )

## Build score

In [5]:
# Country weights are used to determine the score of a country during peace time
COUNTRY_WEIGHTS = {
    'development_cost': 100,               # Each 10% is 10 points
}

# War weights are used to determine the score of a country during war time and to find the best policies
MILITARY_WEIGHTS ={
    'infantry_power': 150,                  # Each 10% is 15 points
    'artillery_power': 150,                 # Each 10% is 15 points
    'discipline': 300,                      # Each 10% is 30 points
}

## Build eval

In [6]:
EVAL_MAIN_EFFECTS = [
    'development_cost',
    'discipline',
    'fire_damage',
    'fire_damage_received',
    'shock_damage',
    'shock_damage_received',
    'land_morale',
    'infantry_power',
    'artillery_power',
]


In [7]:
build = compute_build(ideas=(
    'decentralism', 
    'fem_boy', 
    'development', 
    'trade_ideas', 
    'imperialismus', 
    'standing_army', 
    'quality', 
    'weapon_quality'
))
print_build(build, effects=EVAL_MAIN_EFFECTS)

--------------------------------------------------------------------
Build - score: 232.5 - ideas: ('decentralism', 'fem_boy', 'development', 'trade_ideas', 'imperialismus', 'standing_army', 'quality', 'weapon_quality')
	development_cost: 0.60
	discipline: 0.28
	fire_damage: 0.00
	fire_damage_received: 0.00
	shock_damage: 0.00
	shock_damage_received: 0.00
	land_morale: 0.00
	infantry_power: 0.30
	artillery_power: 0.30


In [9]:
build = compute_build(ideas=(
    'innovativeness_ideas',
    'economic_ideas', 
    'development', 
    'assimilation', 
    'imperialismus', 
    'standing_army', 
    'quality', 
    'weapon_quality'
))
print_build(build, effects=EVAL_MAIN_EFFECTS)

--------------------------------------------------------------------
Build - score: 265.0 - ideas: ('innovativeness_ideas', 'economic_ideas', 'development', 'assimilation', 'imperialismus', 'standing_army', 'quality', 'weapon_quality')
	development_cost: 0.40
	discipline: 0.28
	fire_damage: 0.00
	fire_damage_received: 0.00
	shock_damage: 0.00
	shock_damage_received: 0.05
	land_morale: 0.00
	infantry_power: 0.55
	artillery_power: 0.40


In [10]:
build = compute_build(ideas=(
    'decentralism', 
    'economic_ideas', 
    'katholisch0', 
    'trade_ideas', 
    'imperialismus', 
    'offensive', 
    'fire_ideas', 
    'weapon_quality'
))
print_build(build, effects=EVAL_MAIN_EFFECTS)

--------------------------------------------------------------------
Build - score: 227.5 - ideas: ('decentralism', 'economic_ideas', 'katholisch0', 'trade_ideas', 'imperialismus', 'offensive', 'fire_ideas', 'weapon_quality')
	development_cost: 0.40
	discipline: 0.15
	fire_damage: 0.05
	fire_damage_received: 0.00
	shock_damage: 0.00
	shock_damage_received: 0.07
	land_morale: 0.25
	infantry_power: 0.30
	artillery_power: 0.65


## Allowed idea values

### Admin

- innovativeness_ideas
- economic_ideas
- expansion_ideas
- administrative_ideas
- humanist_ideas
- judiciary
- development
- strong_men
- fem_boy 
- public_admin
- centralism
- decentralism

#### Gov type

- monarchie0
- republik0
- aristo0
- diktatur0
- horde0

#### Religion

- religious_ideas
- katholisch0
- protestant0
- reformiert0
- orthodox0
- islam0
- tengri0
- hindu0
- confuci0
- budda0
- norse0
- shinto0
- cathar0
- coptic0
- romuva0
- suomi0
- jew0
- slav0
- helle0
- mane0
- animist0
- feti0
- zoro0
- ancli0
- nahu0
- mesoam0
- inti0
- tote0
- shia0
- ibadi0
- hussite0
- alche0

### Diplo

- spy_ideas
- dynastic
- influence_ideas
- trade_ideas
- exploration_ideas
- maritime_ideas
- heavy_ship
- galley_ship
- trade_ship
- colonial_emp
- assimilation
- sociaty
- propaganda
- fleet_base
- nationalismus
- konigreich0
- imperialismus

### Military

- offensive
- defensive
- quality
- quantity
- general_staff
- standing_army
- conscription
- merc_army
- weapon_quality
- fortress
- war_production
- formation0
- militarism
- shock_ideas
- fire_ideas